In [1]:
#Utilizacion de Word2Vec 

In [2]:
import pandas as pd
pd.set_option('max_colwidth', -1)
import string
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('../dataset/train.csv')
df_test = pd.read_csv('../dataset/test.csv')
df.drop(columns=['location', 'keyword', 'target'], inplace=True)
df_test.drop(columns=['location', 'keyword'], inplace=True)
df = df.merge(df_test, how='outer')

In [4]:
#Al igual que en otros notebooks, preprocesamiento previo
# Quitamos las urls
df['text'] = df['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['text'] = df['text'].apply(clean_text)

In [66]:
df.head()

,id,text,word_count
0,1,deeds reason earthquake may allah forgive us,7
1,4,forest fire near la ronge sask canada,7
2,5,residents asked shelter place notified officers evacuation shelter place orders expected,11
3,6,13000 people receive wildfires evacuation orders california,8
4,7,got sent photo ruby alaska smoke wildfires pours school,10


In [67]:
#Para que esto funcione, se requiere que todas las palabras de todos los textos de los tweets esten en una lista
palabras = []
def extraer_palabras(string):
    splitted = string.split(' ')
    lista = []
    for palabra in splitted:
        lista.append(palabra)
    palabras.append(lista)
    
df['text'].apply(extraer_palabras)

from gensim.models import Word2Vec

word2vec = Word2Vec(palabras, min_count=2)
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'deeds': <gensim.models.keyedvectors.Vocab object at 0x0000028E0B5170F0>, 'reason': <gensim.models.keyedvectors.Vocab object at 0x0000028E0B8027B8>, 'earthquake': <gensim.models.keyedvectors.Vocab object at 0x0000028E0B802748>, 'may': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7400>, 'allah': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7438>, 'forgive': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7470>, 'us': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE74A8>, 'forest': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE74E0>, 'fire': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7518>, 'near': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7550>, 'la': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE7588>, 'canada': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE75C0>, 'residents': <gensim.models.keyedvectors.Vocab object at 0x0000028E0BBE75F8>, 'asked': <gensim.models.keyedvect

In [70]:
#Ahora, voy a aplicarle a cada una de las palabras, un average para crear la columna puntaje.

def calcular_promedio(text,string):
    split = text.split(' ')
    promedio = 0
    cantidad_de_palabras = 0
    for palabra in split:
        cantidad_de_palabras+=1
        try:
            promedio += word2vec.wv.similarity(string,palabra)
        except KeyError:
            continue
    return promedio/cantidad_de_palabras    
    
df['similitud_debris'] = df['text'].apply(lambda x: calcular_promedio(x,"debris"))
df['similitud_derailment'] = df['text'].apply(lambda x: calcular_promedio(x,"derailment"))
df['similitud_wreckage'] = df['text'].apply(lambda x: calcular_promedio(x,"wreckage"))
df['similitud_outbreak'] = df['text'].apply(lambda x: calcular_promedio(x,"outbreak"))
df['similitud_typhoon'] = df['text'].apply(lambda x: calcular_promedio(x,"typhoon"))
df['similitud_bombing'] = df['text'].apply(lambda x: calcular_promedio(x,"bombing"))

df

,id,text,word_count,similitud_debris,similitud_derailment,similitud_wreckage,similitud_outbreak,similitud_typhoon,similitud_bombing
0,1,deeds reason earthquake may allah forgive us,7,0.995934,0.996335,0.995461,0.993742,0.996298,0.995747
1,4,forest fire near la ronge sask canada,7,0.713841,0.714017,0.713508,0.712389,0.714055,0.713592
2,5,residents asked shelter place notified officers evacuation shelter place orders expected,11,0.908196,0.908499,0.907772,0.906269,0.908535,0.907977
3,6,13000 people receive wildfires evacuation orders california,8,0.997568,0.997858,0.997230,0.995761,0.997921,0.997255
4,7,got sent photo ruby alaska smoke wildfires pours school,10,0.799382,0.799647,0.799015,0.797832,0.799691,0.799226
5,8,rockyfire update california hwy 20 closed directions due lake county fire cafire wildfires,15,0.865596,0.865885,0.865229,0.863757,0.865917,0.865369
6,10,flood disaster heavy rain causes flash flooding streets manitou colorado springs areas,12,0.915847,0.916139,0.915416,0.914009,0.916160,0.915573
7,13,im top hill see fire woods,6,0.995076,0.995555,0.994548,0.993035,0.995633,0.994971
8,14,theres emergency evacuation happening building across street,7,0.999372,0.999718,0.998901,0.997451,0.999782,0.999137
9,15,im afraid tornado coming area,5,0.999117,0.999510,0.998655,0.997054,0.999484,0.998778


In [71]:
df.set_index('id',inplace =True)
df.drop(columns=['text']).to_csv('../features/word2vec.csv', index=False)